In [1]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision 
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms

# os
import os

# Path
from pathlib import Path

#Random
import random
import math

# Download the data using request

Comment out if not used

## Data loading and preproccessing


In [2]:
import os
import shutil
import random

def half_files_randomly(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through each subfolder in the input directory
    for subdir, _, files in os.walk(input_dir):
        # Create a list of file paths in the current subfolder
        file_paths = [os.path.join(subdir, file) for file in files]

        # Calculate the number of files to keep (half of the total)
        num_files_to_keep = len(file_paths) // 2

        # Randomly select files to keep
        files_to_keep = random.sample(file_paths, num_files_to_keep)

        # Create the corresponding subfolder in the output directory
        output_subdir = os.path.join(output_dir, os.path.relpath(subdir, input_dir))
        os.makedirs(output_subdir, exist_ok=True)

        # Move selected files to the output subfolder
        for file_to_keep in files_to_keep:
            output_path = os.path.join(output_subdir, os.path.basename(file_to_keep))
            shutil.move(file_to_keep, output_path)

# Example usage:
input_directory = '/path/to/your/input/directory'
output_directory = '/path/to/your/output/directory'

#half_files_randomly("data/animals_raw_halved", "data/animals_raw_halved_2/")

(Split) - Here we split our data into 80% training data and 20% for testing using a libary called splitfolders, which splits a single directory into a 

In [3]:

# Setup path to data folder
data_path = Path("data/")


In [4]:
#import splitfolders

#splitfolders.ratio(data_path, output="dogdataset", seed=69420, ratio=(.8, 0.2)) 

In [5]:

# Setup path to data folder
#data_path = Path("dogdataset/")
# Setup path to train folder
#data_path_train = Path("dogdataset/train")
# Setup path to test folder
#data_path_test = Path("dogdataset/val")
#Setup all image paths
#all_images = list(Path(os.path.join(data_path)).glob("*/*/*.jpg"))


(Overview) of out train/val picture count

In [6]:
CLASSES = os.listdir(data_path)
CLASSES = sorted(CLASSES)

#remove ds store from the mac. 
if CLASSES[0] == ".DS_Store":
    CLASSES.remove(".DS_Store")

for label in CLASSES:
    total_images_class = list(Path(os.path.join(data_path, label)).glob("*/*.*"))
    print(f'* {label}: {len(total_images_class)} images')

* animals: 0 images
* animals_3_classes: 12782 images
* animals_3_classes_split: 0 images
* animals_5_classes_split: 0 images
* animals_raw: 26179 images
* animals_raw_halved: 6829 images
* animals_raw_halved_2: 6827 images
* animals_raw_halved_2.zip: 0 images
* cifar10: 0 images
* dog_breeds: 0 images
* dogs: 2307 images
* german_signs: 25352 images
* pizza_steak_sushi: 0 images
* translate.py: 0 images


(Dataset helperfunction) Following class is from pytorch documentation, part 04: Custom data sets:https://www.learnpytorch.io/04_pytorch_custom_datasets/#what-is-a-custom-dataset

We modify the documentation code to fit our paths

In [7]:
from typing import Tuple, Dict, List

# Make function to find classes in target directory
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:

    # 1. Get the class names by scanning the target directory
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
    
    # 2. Raise an error if class names not found
    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes in {directory}.")
        
    # 3. Create a dictionary of index labels (computers prefer numerical rather than string labels)
    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
    return classes, class_to_idx


In [8]:
# Write a custom dataset class (inherits from torch.utils.data.Dataset)
from torch.utils.data import Dataset
import pathlib
from PIL import Image

# 1. Subclass torch.utils.data.Dataset
class ImageFolderCustom(Dataset):
    
    # 2. Initialize with a targ_dir and transform (optional) parameter
    def __init__(self, targ_dir: str, transform=None) -> None:
        
        # 3. Create class attributes
        # Get all image paths
        self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg")) # note: you'd have to update this if you've got .png's or .jpeg's
        random.shuffle(self.paths)
        # Setup transforms
        self.transform = transform
        # Create classes and class_to_idx attributes
        self.classes, self.class_to_idx = find_classes(targ_dir)

    # 4. Make function to load images
    def load_image(self, index: int) -> Image.Image:
        "Opens an image via a path and returns it."
        image_path = self.paths[index]
        return Image.open(image_path) 
    
    # 5. Overwrite the __len__() method (optional but recommended for subclasses of torch.utils.data.Dataset)
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(self.paths)
    
    # 6. Overwrite the __getitem__() method (required for subclasses of torch.utils.data.Dataset)
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        "Returns one sample of data, data and label (X, y)."
        img = self.load_image(index)
        class_name  = self.paths[index].parent.name # expects path in data_folder/class_name/image.jpeg
        class_idx = self.class_to_idx[class_name]

        # Convert to RGB if the image is grayscale
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        # Transform if necessary
        if self.transform:
            return self.transform(img), class_idx # return data, label (X, y)
        else:
            return img, class_idx # return data, label (X, y)

(Data transform)

Prepare the data transform using torchvision.transform

The tinyvvg input neoruns, need a 64x64 tensorshape. Therefore we resize the images to 64x64

In [9]:
# Write transform for image
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(64,64)),
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])

Now we can create our datasets:

In [10]:
# Normalize for CIFAR-10
normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
resize = transforms.Resize((64, 64))

# Manual augmentations
manual_tf = transforms.Compose([
    resize,
    transforms.RandomCrop(64, padding=4, padding_mode="reflect"),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

# Auto augmentations
std_tf = transforms.Compose([
    resize,
    transforms.ToTensor(),
    normalize
])

In [11]:
data_path = Path("data/")

# Setup path
animals_path = data_path / "animals_raw_halved_2"

#make into one big data set incase we want to use k-fold cross validation for later. 
WholeDataset = ImageFolderCustom(targ_dir=animals_path, 
                                      transform=manual_tf)

<b> Transform all the data images and load into class folder

## Dataloader

use CPU if not CUDA

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Class module, tinyvgg -> Forwardpass

# TinyVGG Model

Model

## Utility, helper functions

In [13]:
import requests
from pathlib import Path 

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

helper_functions.py already exists, skipping download


## TINYVVG OPTIMIZATION, BACKWARD PASS


SET LOSS AND OPT FUNCTION (HYPER PARAMETER)

In [14]:
from helper_functions import accuracy_fn
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()

### Training and testing functions

In [15]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            test_pred = model(X)
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1)
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")
    return test_acc

## Standard train/test

# Random search

In [16]:
# random search logistic regression model on the sonar dataset
import scipy.stats

hyperparameters = {
    'learning_rate': (1,1), #(random.choice([0.01, 0.001, 0.0001]), 0.0000001),  # Range for learning rate
    'batch_size': (16, 64),         # Range for batch size
    'epochs': (15, 30),             # Range for number of epochs
    'dropout': [0.2, 0.3, 0.4, 0.5],           # Range for dropout rate
    #'weight_decay': (0.0001, 0.01),  # Range for weight decay
    'optimizer': ['adam', 'sgd', 'rmsprop'],
}

In [17]:
def sample_hyperparameters(hyperparameters):
    trialHyperparameters = {}
    
    hyperparameters['learning_rate'] = (random.choice([0.02,0.01,0.001,0.0001]), 0.0000001)

    for param, value in hyperparameters.items():
        #We want to sepperate the learning rate for itself, as we want to use a log uniform, so the values go to the smaller size. 
        if param == 'learning_rate':
            trialHyperparameters[param] = scipy.stats.reciprocal(value[0],value[1])

        #We want to be able to choose between 2 factor numbers in batchsize
        if param == 'batch_size':
            trialHyperparameters[param] = random.choice([16, 32, 64])

        #Give all the parameters with a range a value uniformly between the 2 thresholds. 
        elif isinstance(value, tuple): 
            trialHyperparameters[param] = random.uniform(value[0], value[1])
            #print(trialHyperparameters[param])
        #Give all the parameters with a category a value uniformly between the 2 thresholds. 
        elif isinstance(value, list):  
            trialHyperparameters[param] = random.choice(value)
            
        #Now we want to round the values
        if param == 'epochs':
            trialHyperparameters[param] = round(trialHyperparameters[param])
        if param == 'batch_size':
            trialHyperparameters[param] = round(trialHyperparameters[param])
        if param == 'learning_rate':
            trialHyperparameters[param] = round(trialHyperparameters[param], 5)
    
    return trialHyperparameters

sample_hyperparameters(hyperparameters)

{'learning_rate': 0.001,
 'batch_size': 32,
 'epochs': 19,
 'dropout': 0.3,
 'optimizer': 'rmsprop'}

### Evaluation loss algorithm

In [18]:
# credit: https://stackoverflow.com/a/73704579

import numpy as np

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

### Random search

In [19]:
from scripts import model as m
from scripts import engine
from tqdm.auto import tqdm
from lib.experiment_tracking import create_writer
from torch.utils.tensorboard import SummaryWriter

# Define the objective function
def objective_function(trialHyperparameters):
    
    accuracies = 0
 
    # Perform k-fold cross-validation with 5 folds
    for fold in range(5):
        
        # Print outs
        optim = trialHyperparameters['optimizer']
        lrn = trialHyperparameters['learning_rate']
        drop = trialHyperparameters['dropout']
        bs = trialHyperparameters['batch_size']
        epochs = trialHyperparameters['epochs']

        # Generate the model.
        #model = tinyVGG(input_shape=3, hidden_units=10, output_shape=10).to(device)
        #model = m.TinyVGG(input_shape=3, hidden_units=10, output_shape=10).to(device)
        model = m.AJ_CNN(input_shape=3, hidden_units=10, output_shape=5, dropout=drop).to(device)
        model_name = model.__class__.__name__

        if(trialHyperparameters['optimizer'] == "adam"):
            optimizer = torch.optim.Adam(params=model.parameters(),lr = trialHyperparameters['learning_rate'])
        if(trialHyperparameters['optimizer'] == "sgd"):
            optimizer = torch.optim.SGD(params=model.parameters(),lr = trialHyperparameters['learning_rate'])
        if(trialHyperparameters['optimizer'] == "rmsprop"):
            optimizer = torch.optim.RMSprop(params=model.parameters(),lr = trialHyperparameters['learning_rate'])


        loss_fn = nn.CrossEntropyLoss()

        fold_size = len(WholeDataset) // 5
        
        #print("Fold size: ", fold_size)
        #print("Fold: ", fold)

        # Split the dataset into training and validation sets for each fold
        val_indices = range(fold * fold_size, (fold + 1) * fold_size)
        #print("val_indices: ", val_indices)
        train_indices = [i for i in range(len(WholeDataset)) if i not in val_indices]
        #print("train_indices: ", train_indices)
        
        train_dataset = torch.utils.data.Subset(WholeDataset, train_indices)
        test_dataset = torch.utils.data.Subset(WholeDataset, val_indices)


        trainFold = DataLoader(train_dataset, batch_size=trialHyperparameters['batch_size'], shuffle=True)
        #print(len(trainFold))
        
        #We dont shuffle, as we want to compare them without creating another confounder
        TestFold = DataLoader(test_dataset, batch_size=trialHyperparameters['batch_size'], shuffle=False)
        #print(len(TestFold))
        
        # Create empty results dictionary
        results = {"train_loss": [],
                "train_acc": [],
                "test_loss": [],
                "test_acc": [],
                "lr": []
        }
        
        writer = create_writer(experiment_name=f"{optim}_lr_{lrn:.4f}",
                                    model_name=model_name,
                                    extra=f"bs_{bs}_drop_{drop:.2f}_{epochs}_epochs")
        
        for epoch in tqdm(range(epochs)):
            
            # Early stopper
            if epoch+1 > 7:
                last_five = (results.get("test_loss")[-5:])
                avg_last_five = sum(last_five)/5
                
                if avg_last_five > 1.4:
                    return 0.2
                    #break
            
            train_loss, train_acc, learning_rate = engine.train_step(model=model,
                                           dataloader=trainFold,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        
            test_loss, test_acc = engine.test_step(model=model,
                                            dataloader=TestFold,
                                            loss_fn=loss_fn,
                                            device=device)
            
            print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"lr: {learning_rate:.4f}"
            )
            
            # Update results dictionary
            results["train_loss"].append(train_loss)
            results["train_acc"].append(train_acc)
            results["test_loss"].append(test_loss)
            results["test_acc"].append(test_acc)
            results["lr"].append(learning_rate)
            
            #print(epoch)
            #train_step(data_loader= trainFold,
            #    model=model, 
            #    loss_fn=loss_fn,
            #    optimizer=optimizer,
            #    accuracy_fn=accuracy_fn,
            #    device=device
            #)
            #accuracy = test_step(data_loader= TestFold,
            #    model=model,
            #    loss_fn=loss_fn,
            #    accuracy_fn=accuracy_fn,
            #    device=device
            #)
            
            # Use the writer parameter to track experiments
            if writer: # See if there's a writer, if so, log to it
                # Add results to SummaryWriter
                writer.add_scalars(main_tag="Loss", 
                                tag_scalar_dict={"train_loss": train_loss,
                                                    "test_loss": test_loss},
                                global_step=epoch)
                writer.add_scalars(main_tag="Accuracy", 
                                tag_scalar_dict={"train_acc": train_acc,
                                                    "test_acc": test_acc}, 
                                global_step=epoch)

                # Close the writer
                writer.close()
            else:
                pass

        accuracies += test_acc
    
    mean_acc = accuracies/5
    
    print(f"Mean acc: {mean_acc}")
    return mean_acc

In [20]:
import json
from tqdm.auto import tqdm

def trialRandomInit():
    trials = 100
    bestHyperparameters = {}
    bestHyperparametersAccuracy = 0
    
    def save_results(results):
        with open('results.json', 'a') as file:
            json.dump(results, file)
            file.write('\n')  # Add a newline to separate entries

    # Training of the model.
    for epoch in tqdm(range(trials)):
        newAcc = 0

        # Initialize the values of trial hyperparameters
        trialHyperparameters = sample_hyperparameters(hyperparameters)

        print("Trial", epoch+1, ": ", trialHyperparameters)

        ##Init the accuracy of the current trial 
        newAcc = objective_function(trialHyperparameters)
        
        # Save trial results to a dictionary
        trial_results = {
            'trial_number': epoch+1,
            'trial_hyperparameters': trialHyperparameters,
            'trial_accuracy': newAcc
        }
        
        # Save trial results to the JSON file
        save_results(trial_results)

        ##Check if these parameters do better than last, and if yes, change the best to the best. 
        if(newAcc > bestHyperparametersAccuracy):
            bestHyperparametersAccuracy = newAcc
            bestHyperparameters = trialHyperparameters

    print("Best Hyperparameters:", bestHyperparameters)
    print("Best Accuracy: ", bestHyperparametersAccuracy)

    # Save the best hyperparameters and accuracy to a JSON file
    with open('best_results.json', 'w') as file:
        results = {
            'best_hyperparameters': bestHyperparameters,
            'best_accuracy': bestHyperparametersAccuracy
        }
        json.dump(results, file)

trialRandomInit()

  0%|          | 0/100 [00:00<?, ?it/s]

Trial 1 :  {'learning_rate': 3e-05, 'batch_size': 64, 'epochs': 27, 'dropout': 0.2, 'optimizer': 'sgd'}
[INFO] Created SummaryWriter, saving to: runs/2024-01-16/sgd_lr_0.0000/AJ_CNN/bs_64_drop_0.20_27_epochs...


  0%|          | 0/27 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.5215 | train_acc: 0.4160 | test_loss: 1.5824 | test_acc: 0.3828 | lr: 0.0000
Epoch: 2 | train_loss: 1.5706 | train_acc: 0.3190 | test_loss: 1.5639 | test_acc: 0.4010 | lr: 0.0000
Epoch: 3 | train_loss: 1.6057 | train_acc: 0.3405 | test_loss: 1.5732 | test_acc: 0.3568 | lr: 0.0000
Epoch: 4 | train_loss: 1.6145 | train_acc: 0.3307 | test_loss: 1.5473 | test_acc: 0.3568 | lr: 0.0000
Epoch: 5 | train_loss: 1.5677 | train_acc: 0.3249 | test_loss: 1.5184 | test_acc: 0.4010 | lr: 0.0000
Epoch: 6 | train_loss: 1.5828 | train_acc: 0.3366 | test_loss: 1.4993 | test_acc: 0.3490 | lr: 0.0000
Epoch: 7 | train_loss: 1.6110 | train_acc: 0.3327 | test_loss: 1.5003 | test_acc: 0.3906 | lr: 0.0000
Trial 2 :  {'learning_rate': 6e-05, 'batch_size': 32, 'epochs': 18, 'dropout': 0.2, 'optimizer': 'sgd'}
[INFO] Created SummaryWriter, saving to: runs/2024-01-16/sgd_lr_0.0001/AJ_CNN/bs_32_drop_0.20_18_epochs...


  0%|          | 0/18 [00:00<?, ?it/s]

...

Epoch: 1 | train_loss: 1.5486 | train_acc: 0.2743 | test_loss: 1.4804 | test_acc: 0.3125 | lr: 0.0000
Epoch: 2 | train_loss: 1.3337 | train_acc: 0.3403 | test_loss: 1.3337 | test_acc: 0.3828 | lr: 0.0000
Epoch: 3 | train_loss: 1.2403 | train_acc: 0.3771 | test_loss: 1.2474 | test_acc: 0.3047 | lr: 0.0000
Epoch: 4 | train_loss: 1.2328 | train_acc: 0.3493 | test_loss: 1.1978 | test_acc: 0.3594 | lr: 0.0000
Epoch: 5 | train_loss: 1.1721 | train_acc: 0.4097 | test_loss: 1.1927 | test_acc: 0.3984 | lr: 0.0000
Epoch: 6 | train_loss: 1.1971 | train_acc: 0.4160 | test_loss: 1.1624 | test_acc: 0.3828 | lr: 0.0000
Epoch: 7 | train_loss: 1.1387 | train_acc: 0.4285 | test_loss: 1.1610 | test_acc: 0.3828 | lr: 0.0000
Epoch: 8 | train_loss: 1.1274 | train_acc: 0.4507 | test_loss: 1.1199 | test_acc: 0.4375 | lr: 0.0000
Epoch: 9 | train_loss: 1.1507 | train_acc: 0.4222 | test_loss: 1.1235 | test_acc: 0.4531 | lr: 0.0000
Epoch: 10 | train_loss: 1.1104 | train_acc: 0.4340 | test_loss: 1.1066 | test_acc:

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.3372 | train_acc: 0.3438 | test_loss: 1.3755 | test_acc: 0.3750 | lr: 0.0000
Epoch: 2 | train_loss: 1.1897 | train_acc: 0.3750 | test_loss: 1.2054 | test_acc: 0.4688 | lr: 0.0000
Epoch: 3 | train_loss: 1.1177 | train_acc: 0.4403 | test_loss: 1.1184 | test_acc: 0.5078 | lr: 0.0000
Epoch: 4 | train_loss: 1.1789 | train_acc: 0.3792 | test_loss: 1.0453 | test_acc: 0.6016 | lr: 0.0000
Epoch: 5 | train_loss: 1.0775 | train_acc: 0.4736 | test_loss: 1.0711 | test_acc: 0.5156 | lr: 0.0000
Epoch: 6 | train_loss: 1.0907 | train_acc: 0.4861 | test_loss: 1.0509 | test_acc: 0.5234 | lr: 0.0000
Epoch: 7 | train_loss: 1.0900 | train_acc: 0.4660 | test_loss: 1.0149 | test_acc: 0.5703 | lr: 0.0000
Epoch: 8 | train_loss: 1.0656 | train_acc: 0.4694 | test_loss: 1.0513 | test_acc: 0.5781 | lr: 0.0000
Epoch: 9 | train_loss: 1.0619 | train_acc: 0.4986 | test_loss: 1.0385 | test_acc: 0.5547 | lr: 0.0000
Epoch: 10 | train_loss: 1.0545 | train_acc: 0.4861 | test_loss: 1.0199 | test_acc:

  0%|          | 0/23 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.3510 | train_acc: 0.3694 | test_loss: 1.4193 | test_acc: 0.4141 | lr: 0.0000
Epoch: 2 | train_loss: 1.1713 | train_acc: 0.4688 | test_loss: 1.3041 | test_acc: 0.3750 | lr: 0.0000
Epoch: 3 | train_loss: 1.1550 | train_acc: 0.4035 | test_loss: 1.2543 | test_acc: 0.3125 | lr: 0.0000
Epoch: 4 | train_loss: 1.1202 | train_acc: 0.4549 | test_loss: 1.1858 | test_acc: 0.3828 | lr: 0.0000
Epoch: 5 | train_loss: 1.1354 | train_acc: 0.4292 | test_loss: 1.1848 | test_acc: 0.4531 | lr: 0.0000
Epoch: 6 | train_loss: 1.0761 | train_acc: 0.4597 | test_loss: 1.1639 | test_acc: 0.4453 | lr: 0.0000
Epoch: 7 | train_loss: 1.1012 | train_acc: 0.4368 | test_loss: 1.1598 | test_acc: 0.4453 | lr: 0.0000
Epoch: 8 | train_loss: 1.0878 | train_acc: 0.4903 | test_loss: 1.1345 | test_acc: 0.4297 | lr: 0.0000
Epoch: 9 | train_loss: 1.0635 | train_acc: 0.4861 | test_loss: 1.1151 | test_acc: 0.5000 | lr: 0.0000
Epoch: 10 | train_loss: 1.0963 | train_acc: 0.4500 | test_loss: 1.0963 | test_acc:

In [ ]:
%%script false --no-raise-error

def trialRandomInit():
    trials = 5
    bestHyperparameters = []
    bestHyperparametersAccuracy = 0

    # Training of the model.
    for epoch in range(trials):
        newAcc = 0

        # Initialize the values of trial hyper parameters
        trialHyperparameters = sample_hyperparameters(hyperparameters)

        print("Trial", epoch, ": ", trialHyperparameters)

        ##Init the accruacy of current trial 
        newAcc = objective_function(trialHyperparameters)

        ##Check if these parameters do better than last, and if yes, change the best to the best. 
        if(newAcc > bestHyperparametersAccuracy):
            bestHyperparametersAccuracy = newAcc
            bestHyperparameters = trialHyperparameters

    print(bestHyperparameters)
    print("acc: ", bestHyperparametersAccuracy )

trialRandomInit()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# Grid search

In [ ]:
hyperparameters = {
    'learning_rate': [0.001, 0.0001],  # Range for learning rate
    'batch_size': [32, 64],         # Range for batch size
    'epochs': (20),               # Range for number of epochs
    'dropout': [0.3, 0.5],           # Range for dropout rate
    #'weight_decay': (0.0001, 0.01),  # Range for weight decay
    'optimizer': ['adam', 'sgd', 'rmsprop'],
}

hyperparametersTest = {
    'learning_rate': [0.0001],  # Range for learning rate
    'batch_size': [32],         # Range for batch size
    'epochs': [10],               # Range for number of epochs
    'optimizer': ['adam', 'sgd', 'rmsprop'],
}

In [ ]:
def trialGridInit():

    bestHyperparameters = []
    bestHyperparametersAccuracy = 0

    # Training of the model.
    for lr in hyperparametersTest['learning_rate']:
        for batch_size in hyperparametersTest['batch_size']:
            for epochs in hyperparametersTest['epochs']:
                for optimizer in hyperparametersTest['optimizer']:
                    newAcc = 0

                    # Initialize the values of trial hyper parameters
                    trialHyperparameters = {}
                    trialHyperparameters = {'learning_rate': lr, 'batch_size': batch_size, 'epochs':epochs, 'optimizer':optimizer}

                    print("Trial",": ", trialHyperparameters)

                    ##Init the accruacy of current trial 
                    newAcc = objective_function(trialHyperparameters)

                    ##Check if theese parameters do better than last, and if yes, change the best to the best. 
                    if(newAcc > bestHyperparametersAccuracy):
                        bestHyperparametersAccuracy = newAcc
                        bestHyperparameters = trialHyperparameters

    print(bestHyperparameters)
    print("acc: ", bestHyperparametersAccuracy )

trialGridInit()